# Toggle Time Tracking

## Configuration

## Setup

### Import sub-modules

In [7]:
# Import required sub-modules

# python
import sys
import os
import enum
import datetime
from dateutil.parser import parse
import time
import pytz
import json

# iPython
import IPython
from IPython.display import display
from IPython.display import Image

# pandas
import pandas as pd

# numpy
import numpy as np

# plotly
import plotly as ply
import plotly.graph_objs as go
import plotly.figure_factory as ff
ply.offline.init_notebook_mode(connected=True)
import plotly.io as pio

# Toggl Wrapper API 
# https://github.com/matthewdowney/TogglPy
import TogglPy

# Verbose what we are working with
print("python: {}".format(sys.version))
print("    - os")
print("    - datetime")
print("    - enum")

print("ipython {}".format(IPython.__version__))
print("pandas: {}".format(pd.__version__))
print("numpy: {}".format(np.__version__))
print("plotly: {}".format(ply.__version__))

ModuleNotFoundError: No module named 'TogglPy'

### Configuration

In [ ]:
# Setup local input directories
inputDir = "input/"
if (os.path.exists(inputDir)) is False:
    os.makedirs(inputDir)
if (os.path.isdir(inputDir)) is False:
    raise NotADirectoryError("{} is not a directory".format(inputDir))

In [ ]:
# Setup local input directories
outputDir = "output/"
if (os.path.exists(outputDir)) is False:
    os.makedirs(outputDir)
if (os.path.isdir(outputDir)) is False:
    raise NotADirectoryError("{} is not a directory".format(outputDir))

In [ ]:
# Graph output Options
class GraphOutputOption(enum.Enum):
    none = 'none'                     # Do not generate any plots
    inline = 'inline'                 # Generate inline plots only
    htmlFile = 'htmlFile'             # Generate plots in external HTML files
    both = 'both'                     # Generate plots both inline and in external html files
    
class GraphInteractionOption(enum.Enum):
    static = 'static'                 # Generate static inline plots (as images)
    interactive = 'interactive'       # Generate interactive inline plots

notebookGraphingInteraction = GraphInteractionOption('interactive')
notebookGraphingOutputs = GraphOutputOption('both')

staticImageSize = {'width':1000, 'height':500, 'scale':1}

GraphAutoOpenHTML = False              # Auto open external HTML files [True/False]

In [ ]:
# Pandas output options
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 50)

### Common Functions

In [ ]:
# Plot figures
def plot_figure(graphFilename, fig, overrideNotebookGraphingInteraction=None):
    """Creates the graph plots depenmding on the choosen option

    Args:
        graphFilename: Filename of the output html-file in case of Option htmlfile
        fig: ply.graph_objs.Figure(data=traces, layout=mylayout)
        overrideNotebookGraphingInteraction: overrides graphic output for this figure
    Returns:
        None
    Raises:
        None

    """
    if (notebookGraphingOutputs == GraphOutputOption('htmlFile')) or (notebookGraphingOutputs == GraphOutputOption('both')):
        ply.offline.plot(fig, filename=graphFilename, auto_open=GraphAutoOpenHTML)
    if (notebookGraphingOutputs == GraphOutputOption('inline')) or (notebookGraphingOutputs == GraphOutputOption('both')):
        choice = None
        if overrideNotebookGraphingInteraction == None:
            if notebookGraphingInteraction == GraphInteractionOption('static'):
                choice = 'static'
            elif notebookGraphingInteraction == GraphInteractionOption('interactive'):
                choice = 'interactive'
        elif overrideNotebookGraphingInteraction == GraphInteractionOption('static'):
            choice = 'static'
        elif overrideNotebookGraphingInteraction == GraphInteractionOption('interactive'):
            choice = 'interactive'
        else:
            choice = 'static'
            
        if choice == 'static':
            plot_img = pio.to_image(fig, format='png', width=staticImageSize['width'], height=staticImageSize['height'], scale=staticImageSize['scale'])
            display(Image(plot_img))
        else:
            ply.offline.iplot(fig)
            

## Data Import

In [ ]:
import json

with open("toggl_credentials.json", "r") as file:
    credentials = json.load(file)
    toggl_cr = credentials['toggl']
    APIKEY = toggl_cr['APIKEY']

toggl = TogglPy.Toggl()
toggl.setAPIKey(APIKEY)

### Get toggle information

In [ ]:
toggl_time_entries_url = "https://www.toggl.com/api/v8/time_entries"
toggl_user_url = "https://www.toggl.com/api/v8/me"
toggl_workspaces_url = "https://www.toggl.com/api/v8/workspaces"
toggl_clients_url = "https://www.toggl.com/api/v8/clients"

### User Info

In [ ]:
user = pd.DataFrame.from_dict(toggl.request(toggl_user_url))
user

### Workspaces Info

In [ ]:
workspaces_list = pd.DataFrame.from_dict(toggl.request(toggl_workspaces_url))
workspaces_list

workspaces = pd.DataFrame()
for wid in workspaces_list['id']:
    workspaces = pd.DataFrame.from_dict(toggl.request(toggl_workspaces_url + "/" + str(wid) + "/clients"))
workspaces

### User Client Info

In [ ]:
user_clients = pd.DataFrame.from_dict(toggl.request(toggl_clients_url))
user_clients

### Projects Info

In [ ]:
projects = pd.DataFrame()
for i in list(range(0, len(workspaces_list))):
    projects_list = toggl.request(toggl_workspaces_url + "/" + str(workspaces_list['id'][i]) + "/projects")
    projects_df_temp = pd.DataFrame.from_dict(projects_list)
    projects = pd.concat([projects_df_temp, projects])
projects

### Time Entries

#### Some standart functions

In [ ]:

def fill_clean_timeentries(df, projects, user_clients):
    # Add project name and client name to dataframe
    df['project'] = df.pid.map(projects.set_index('id')['name'].to_dict())
    projects['client'] = projects.cid.map(user_clients.set_index('id')['name'].to_dict())
    df['client'] = df.pid.map(projects.set_index('id')['client'].to_dict())
    # drop unused columns
    drop_columns = ['at', 'billable', 'duronly', 'guid', 'id', 'pid', 'uid', 'wid']
    df = df.drop(drop_columns, axis=1)
    # rearrange columns
    df = df[['client', 'project', 'description', 'start', 'stop', 'duration']]
    # calculate timedelta
    df['duration'] = pd.to_timedelta(df['duration'], unit='sec')
    # transform to datetime
    df['start'] = pd.to_datetime(df['start'])
    df['stop'] = pd.to_datetime(df['stop'])
    
    return df
    
#latest_time_entries = pd.DataFrame.from_dict(toggl.request(toggl_time_entries_url))
#latest_time_entries = fill_clean_timeentries(latest_time_entries, projects, user_clients)
#latest_time_entries.head

In [ ]:
def get_join_date():
    join_date = parse(user['data']['created_at'])
    join_date = join_date.replace(tzinfo=None)
    return join_date

def get_today_date():
    today = datetime.datetime.now()
    return today

def get_last_dom(any_day):
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)

def get_first_dom(any_day):
    any_day = any_day.replace(day=1)
    return any_day

def get_month(addMonth = 0):
    if addMonth > 0:
        date = get_today_date() + datetime.timedelta(addMonth*365/12)
    elif addMonth < 0:
        date = get_today_date() - datetime.timedelta(-1*addMonth*365/12)
    else:
        date = get_today_date()
    startdate = get_first_dom(date)
    enddate = get_last_dom(date)
    return startdate, enddate

def get_week(date):
    start = date - timedelta(days=date.weekday())
    end = start + timedelta(days=6)
    return (start, end)

(start, end) = get_week(get_today_date())
print(start)
print(end)
    
# Function that turns datetimes back to strings since that's what the API likes
def date_only(datetimeVal):
    datePart = datetimeVal.strftime("%Y-%m-%d")
    return datePart

def filterByMonth(df, addMonth = 0):
    startdate, enddate = get_month(addMonth)
    #startdate = get_first_dom(date)
    #enddate = get_last_dom(date)
    filteredTimeEntriesDf = df[(df['start'] > startdate.strftime('%Y-%m-%d')) & (df['start'] < enddate.strftime('%Y-%m-%d'))]
    return filteredTimeEntriesDf, startdate, enddate
    #timeEntriesDf[(timeEntriesDf['start'] > '2019-03-23 07:30:00') & (timeEntriesDf['start'] < '2019-04-23 09:00:00')]
    #timeEntriesDf[(timeEntriesDf['start'] > '2019-03-23') & (timeEntriesDf['start'] < '2019-04-23')]
    
def filterByWeek(df, date):
    startdate, enddate = get_week(date)
    filteredTimeEntriesDf = df[(df['start'] > startdate.strftime('%Y-%m-%d')) & (df['start'] < enddate.strftime('%Y-%m-%d'))]
    return filteredTimeEntriesDf, startdate, enddate

(df, start, end) = filterByWeek(get_today_date())


In [ ]:
# Extract Timelogs Between Two Dates and Export to a CSV
def toggl_timelog_extractor(input_date1, input_date2, save_csv=False, outputdir='output'):
    date1 = parse(input_date1).isoformat() + '+00:00'
    date2 = parse(input_date2).isoformat() + '+00:00'
    param = {
        'start_date': date1,
        'end_date': date2,
    } 
    try:
        temp_log =  pd.DataFrame.from_dict(toggl.request(toggl_time_entries_url, parameters=param))
        if save_csv:
            if not os.path.exists(outputdir + '/detailed/'):
                os.makedirs(outputdir + '/detailed/')
            temp_log.to_csv(outputdir + '/detailed/toggl-time-entries-' + input_date1 + '.csv')
    except: 
        # try again if there is an issue the first time
        temp_log =  pd.DataFrame.from_dict(toggl.request(toggl_time_entries_url, parameters=param))
        if save_csv:
            if not os.path.exists(outputdir + '/daily-detailed/'):
                os.makedirs(outputdir + '/detailed/')
            temp_log.to_csv(outputdir + '/detailed/toggl-time-entries-' + input_date1 + '.csv')
    return temp_log

#### Get all time entries

In [ ]:
date1 = date_only(get_join_date())
date2 = date_only(get_today_date())
timeEntriesDf = toggl_timelog_extractor(date1, date2)
timeEntriesDf = fill_clean_timeentries(timeEntriesDf, projects, user_clients)

In [ ]:
# Filter by date or month
(filteredTimeEntiresDf, startdate, enddate) = filterByMonth(timeEntriesDf, addMonth) # -1 = previous month
#filteredTimeEntiresDf

## Analytics

### Parameter Unique Values

In [ ]:
dfSize = filteredTimeEntiresDf.shape
paramList = list(filteredTimeEntiresDf)
print("| {:>11} | {:13} | {:10} | ".format("Parameter", "Unique Values", "Total Rows"))
print("+{}+{}+{}+".format(13*"-", 15*"-", 12*"-"))
for param in paramList:
  uniqueEntries = filteredTimeEntiresDf[param].unique()
  uniqueEntryOccurence = uniqueEntries.shape[0]
  print("| {:>11} | {:13} | {:10} | ".format(param, uniqueEntryOccurence, dfSize[0]))

### Summary of Working Hours

In [ ]:
# Split Dataframe by Clients
uniqueEntries = filteredTimeEntiresDf['client'].unique()
clientDfs = []
for uniqueEntry in uniqueEntries:
    clientDfs.append(filteredTimeEntiresDf[filteredTimeEntiresDf['client'] == uniqueEntry])     

In [ ]:
print("Working Hours per Clients and Project")
print("  Timeframe: {} => {}".format(startdate.strftime('%d-%m-%Y'), enddate.strftime('%d-%m-%Y')))
print("-------------------------------------")
for i in range(len(clientDfs)):
    print("Client {}".format(uniqueEntries[i]))
    print("------------------")
    clientDfs[i]['duration_hours'] = (clientDfs[i]['duration'].dt.total_seconds() / 3600)
    print(clientDfs[i].groupby('project')['duration_hours'].sum())
    print()

## Visualizations

### Working Hours Pie Chart

In [ ]:
title = "Work by Clients"
labels = filteredTimeEntiresDf['client']
values = filteredTimeEntiresDf['duration']
trace1 = go.Pie(title=title,
                labels=labels,
                values=values,
                hoverinfo='label+percent+value',
                domain=dict(x=[0,0.5]))

title = "Work by Projects"
labels = filteredTimeEntiresDf['project']
values = filteredTimeEntiresDf['duration']
trace2 = go.Pie(title=title,
                labels=labels,
                values=values,
                hoverinfo='label+percent+value',
                domain=dict(x=[0.5,1.0]))

title = "{} - {} Work partition Pie".format(startdate.strftime('%Y-%m-%d'), enddate.strftime('%Y-%m-%d'))
layout = go.Layout(title=title,
                   #annotations=[ann1,ann2],
                   # Hide legend if you want
                   #showlegend=False
                   )

data = [trace1, trace2]
# Create fig with data and layout
fig = go.Figure(data=data,layout=layout)

graphFilename = outputDir + title + '.html'
plot_figure(graphFilename, fig)


### Working Hours Bar Chart

In [ ]:
title = "{} - {} Work Partition Bar".format(startdate.strftime('%Y-%m-%d'), enddate.strftime('%Y-%m-%d'))
clientDfs[0]['duration'] = clientDfs[0]['duration'].apply(lambda x: pd.to_timedelta(x, unit='h'))
labels = clientDfs[0]['project']
values = clientDfs[0]['duration']
trace1 = go.Bar(name=uniqueEntries[0],
                x=labels,
                y=values
                #hoverinfo='label+percent+value'
                #domain=dict(x=[0,0.5])
               )

labels = clientDfs[1]['project']
values = clientDfs[1]['duration']
trace2 = go.Bar(name=uniqueEntries[1],
                x=labels,
                y=values
                #hoverinfo='label+percent+value',
                #domain=dict(x=[0.5,1])
               )

#title = "Work partition from {} to {}".format(startdate.strftime('%d-%m-%Y'), enddate.strftime('%d-%m-%Y'))
layout = go.Layout(title=title,
                   #annotations=[ann1,ann2],
                   # Hide legend if you want
                   #showlegend=False
                   )

data = [trace1, trace2]
# Create fig with data and layout
fig = go.Figure(data=data, layout=layout)

graphFilename = outputDir + title + '.html'
plot_figure(graphFilename, fig)
